number to test
1
4
9
15
19

In [47]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import joblib
import os

def QNNs_gen(data, 
             target_column, 
             feature_column, 
             save_path, 
             number_of_quantile_one_end=9, 
             layer_configs=[[ 256,128,64]], #[128, 64, 32], 
             learning_rates=[0.01], # 0.005, 0.001
             optimizer_classes=[optim.Adam,#optim.AdamW,
                                optim.RMSprop#,
                                #optim.Adagrad,
                                #optim.Adadelta,
                                #optim.Adamax
                                ]
            ):
    def ensure_directory_exists(path):
        if not os.path.exists(path):
            os.makedirs(path)
            print(f"Created directory: {path}")
        else:
            print(f"Directory already exists: {path}")
    ensure_directory_exists(save_path)

    class NeuralNet(nn.Module):
        def __init__(self, input_size, output_size, layers):
            super(NeuralNet, self).__init__()
            self.layers = nn.ModuleList()
            self.activations = nn.ModuleDict()
            last_size = input_size
            for i, size in enumerate(layers):
                self.layers.append(nn.Linear(last_size, size))
                self.activations[f'layer{i+1}'] = nn.ReLU()
                last_size = size
            self.output_layer = nn.Linear(last_size, output_size)

        def forward(self, x):
            for i, linear in enumerate(self.layers):
                x = linear(x)
                x = self.activations[f'layer{i+1}'](x)
            return self.output_layer(x)
    # Data cleaning - Handling NaNs
    data = data.dropna()  # or data.fillna(method='ffill') depending on the requirement

    #data = data.sample(n=5000, random_state=42)  # 'n' is the number of items from the axis to return.


    features = data[feature_column]
    target = data[target_column]
    X = torch.tensor(features.values, dtype=torch.float32)
    y = torch.tensor(target.values, dtype=torch.float32).unsqueeze(1)
    scaler = StandardScaler()
    # Split data into train and temporary test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

    # Split the temporary test set into final test and left-out sets
    X_test, X_left_out, y_test, y_left_out = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
    X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_dataset = TensorDataset(X_test,y_test)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

    best_model_performance = float('inf')
    best_models = None
    # For the bottom quantile from 0 to 0.5 divided by 'number_of_quantile_one_end'
    bottom_quantiles = [(i + 1) / (number_of_quantile_one_end * 2+2) for i in range(number_of_quantile_one_end)]
    # For the top quantile from 0.5 to 1 divided by 'number_of_quantile_one_end'
    top_quantiles = [0.5 + (i + 1) / (number_of_quantile_one_end * 2+2) for i in range(number_of_quantile_one_end)]
    # Sort bottom quantile in descending order
    bottom_quantiles = sorted(bottom_quantiles, reverse=True)
    # Sort top quantile in ascending order (it's already ascending but included for clarity)
    top_quantiles = sorted(top_quantiles)

    # Combine quantiles starting from 0.5 and interleaving top and bottom quantiles
    quantiles = [0.5]
    for t, b in zip(top_quantiles, bottom_quantiles):
        quantiles.append(t)
        quantiles.append(b)
    total_iter=len(layer_configs)*len(learning_rates)*len(optimizer_classes)
    itera=0

    for layer_sizes in layer_configs:
        for lr in learning_rates:
            for Optimizer in optimizer_classes:
            
                models = [NeuralNet(X_train.shape[1], 1, layer_sizes) for _ in quantiles]
                optimizers = [Optimizer(model.parameters(), lr=lr) for model in models]

      
                for epoch in range(100):
                    for inputs, labels in train_loader:
                        current_outputs = {}
                        for optimizer in optimizers:
                            optimizer.zero_grad()
                        for q_idx, model in enumerate(models):
                            outputs = model(inputs)
                            # Check and handle NaNs right after model output
                            if torch.isnan(outputs).any():
                                print(f"NaN detected in outputs during training for quantile {quantiles[q_idx]}")
                            current_outputs[quantiles[q_idx]] = outputs.detach().clone()
                            

                            lower_bound, upper_bound=set_bounds(top_quantiles, bottom_quantiles, current_outputs, quantiles[q_idx])
                            loss = adjusted_quantile_loss(outputs, labels, quantiles[q_idx], lower_bound, upper_bound)
                            loss.backward(retain_graph=True if q_idx < len(quantiles) - 1 else False)
                            # Apply gradient clipping
                            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                        for optimizer in optimizers:
                            optimizer.step()


                # Initialize dictionary to hold R-squared sums and count of batches per quantile
                # Evaluate and print R-squared after training is complete
                total_loss = 0
                # Initialize dictionary to hold R-squared sums and count of batches per quantile
                r2_sums = {q: 0.0 for q in quantiles}
                r2_counts = {q: 0 for q in quantiles}
                total_loss = 0
                with torch.no_grad():
                    for inputs, labels in test_loader:  # Corrected to use test_loader
                        current_outputs = {}
                        for q_idx, model in enumerate(models):
                            model.eval()  # Ensure model is in evaluation mode
                            outputs = model(inputs)
                            if torch.isnan(outputs).any():
                                print(f"NaN detected in model outputs for quantile {quantiles[q_idx]} during evaluation")
                            current_outputs[quantiles[q_idx]] = outputs.detach()

                            labels_np = labels.detach().cpu().numpy()
                            outputs_np = outputs.detach().cpu().numpy()

                            # Safely calculate R-squared, handling potential NaN scenarios
                            if not (np.isnan(labels_np).any() or np.isnan(outputs_np).any()):
                                r2 = r2_score(labels_np, outputs_np)
                                r2_sums[quantiles[q_idx]] += r2
                                r2_counts[quantiles[q_idx]] += 1
                            else:
                                print("NaN found in labels or outputs, skipping R2 calculation")

                            # Calculate loss if needed for the particular quantile
                            lower_bound, upper_bound = set_bounds(top_quantiles, bottom_quantiles, current_outputs, quantiles[q_idx])
                            loss = adjusted_quantile_loss(outputs, labels, quantiles[q_idx], lower_bound, upper_bound)
                            total_loss += loss.item()

                        model.train()  # Reset models to training mode if further training is needed

                # Calculate average R-squared per quantile
                avg_r2_per_quantile = {q: (r2_sums[q] / r2_counts[q] if r2_counts[q] > 0 else float('nan')) for q in quantiles}
                itera += 1
                print(f'Percentage done, {itera/total_iter * 100:.2f}%, config: layer_sizes {layer_sizes}, lr {lr}, Optimizer {Optimizer.__name__}')
                for q, avg_r2 in avg_r2_per_quantile.items():
                    print(f"Average R2 for Quantile {q}: {avg_r2:.4f}")




                # Reset outputs to clear memory after usage
                current_outputs.clear()

                # Check if current configuration is the best based on loss
                #avg_r2 = total_r2 / (len(train_loader) * len(models))
                avg_loss = total_loss / len(train_loader)
                if avg_loss < best_model_performance:
                    best_model_performance = avg_loss
                    best_models = [model.state_dict() for model in models]

                # Save the best models for each quantile
                if best_models:
                    np.save(os.path.join(save_path, 'layer_sizes.npy'), layer_sizes)
                    np.save(os.path.join(save_path, 'quantiles.npy'), quantiles)
                    scaler_file = os.path.join(save_path, 'scaler.pkl')
                    joblib.dump(scaler, scaler_file)
        
                    for idx, model_state in enumerate(best_models):
                        torch.save({
                            'state_dict': model_state,
                            'weights': [param.detach().numpy() for param in models[idx].parameters()]
                        }, os.path.join(save_path, f'model_quantile_{quantiles[idx]:.2f}.pth'))
                




    return quantiles, models, scaler


def set_bounds(top_quantiles, bottom_quantiles, current_outputs, quantile):
    lower_bound = None
    upper_bound = None

    # Check if the quantile is in the top or bottom array
    if quantile in top_quantiles:
        # Find the index of the quantile in the top array
        index = top_quantiles.index(quantile)
        # Set lower bound from the previous quantile or 0.5 if it is the first quantile in the array
        lower_bound = current_outputs.get(top_quantiles[index - 1] if index > 0 else 0.5)

    elif quantile in bottom_quantiles:
        # Find the index of the quantile in the bottom array
        index = bottom_quantiles.index(quantile)
        # Set upper bound from the previous quantile or 0.5 if it is the first quantile in the array
        upper_bound = current_outputs.get(bottom_quantiles[index - 1] if index > 0 else 0.5)

    return lower_bound, upper_bound


# Example usage remains as previously described.
def adjusted_quantile_loss(outputs, targets, quantile, lower_bound, upper_bound, expectile=False):
    errors = targets - outputs

    if expectile:
        # Calculate expectile loss
        squared_errors = errors ** 2
        loss = torch.where(errors > 0, quantile * squared_errors, (1 - quantile) * squared_errors)
    else:
        # Calculate quantile loss
        basic_loss = torch.max( quantile * errors, (quantile - 1) * errors)
        loss = torch.mean((basic_loss))

    # Aggregate loss to a scalar if not already (important for expectile branch)
    if not isinstance(loss, torch.Tensor) or loss.dim() != 0:
        loss = torch.mean(loss)

    # Apply lower and upper bounds penalties
    if lower_bound is not None:
        loss += torch.mean(torch.relu(lower_bound - outputs))
    if upper_bound is not None:
        loss += torch.mean(torch.relu(outputs - upper_bound))

    return loss


# Usage example
#data = pd.read_csv('/home/yui/Downloads/read_and_play/p122_synthetic (2).csv')  # specify the path to your dataset
data = pd.read_csv('/home/yui/Downloads/read_and_play/p122_synthetic_1_1.1_1.2.csv')  # specify the path to your dataset
feature_column = ['battery_2#p122','gas-cc#p122','upv#p122','wind-ons#p122','scen']
target_column = 'value'
save_path = '/home/yui/Downloads/read_and_play/model_output'
# Example usage
quantiles, models, scaler = QNNs_gen(data, target_column, feature_column, save_path, number_of_quantile_one_end=7)


Directory already exists: /home/yui/Downloads/read_and_play/model_output


KeyboardInterrupt: 